In [117]:
import numpy as np
import pandas as pd
import scipy
from scipy.linalg import cholesky
from numpy import mat
from numpy import diag
import math
import time
import random

### cholesky矩阵分解

In [10]:
'''
mchol函数将对称方阵分解为一个下三角矩阵乘以该矩阵转置的形式,函数返回值为下三角矩阵
#输入：欲分解的矩阵x
#输出：cholesky分解所得矩阵L

numpy.linalg和scipy.linalg中都有cholesky（）函数

不同之处在于：

numpy : s = LLT，返回的L是下三角阵
scipy : s = LLT，返回的L是上三角阵

def mchol(x):
    m=np.shape(x)[0]
    n=np.shape(x)[1]
    #检验x是否为方阵
    if m!=n:
        print("Wrong dimensions of matrix!")
    #检验x是否为对称阵
    if sum(sum(x.T !=x))>0:
        print("Input matrix is not symmetrical!")
    return np.linalg.cholesky(x)
"""

### 解L*x =b

In [244]:
###mforwardsolve函数求解线性方程租Lx=b，其中L为下三角矩阵
def mforwardsolve(L,b):
    m=np.shape(L)[0]
    n=np.shape(L)[1]
    #0向量记录求解结果
    x=np.zeros(m)
    #循环每进行一次,求解一个x中的元素，看作矩阵L向量x向量b的维数减一
    #故下述将矩阵L的第i列记为当前矩阵L第一列，将向量x向量b的第i个元素记为当前向量第一个元素
    for i in range(m):
        #求当前循环中x的第一个元素
        x[i]=b[i]/L[i][i]
        #降维后的b向量为原来位置上的元素减去当前矩阵L的第一列的乘积
        if i<m-1:
            b[i+1:]=b[i+1:]-x[i]*L[i+1:,i]
    return x

###mbacksolve函数求解线性方程租Lx=b，其中L为上三角矩阵
#输入：上三角矩阵L，向量b
#输出：线性方程组的解x
def mbacksolve(L,b):
    m=np.shape(L)[0]
    n=np.shape(L)[1]
    x=np.zeros(m)
    #循环每进行一次,求解一个x中的元素，看作矩阵L向量x向量b的维数减一
    #故下述将矩阵L的第i列记为当前矩阵L最后一列，将向量x向量b的第i个元素记为当前向量最后一个元素
    for i in range(m-1,-1,-1):
        x[i]=b[i]/L[i][i]
        if i >0:
            b[:i-1]=b[:i-1]-x[i]*L[:i-1,i]
    return x  

In [213]:
#测试向前和回代法函数
y=np.random.rand(5,4)
x1=np.dot(y.T,y)
b=np.arange(4)

In [253]:
mforwardsolve(np.linalg.cholesky(x1),b)

array([0.        , 1.50913349, 1.81457882, 0.        ])

In [259]:
mbacksolve(scipy.linalg.cholesky(x1),b)

array([-0.75144761,  1.50913349,  1.81457882,  0.        ])

### 岭回归 

In [161]:
def ridgereg(lam,x,y):
    n=np.shape(x)[0]#行数
    p=np.shape(x)[1]#维数
    
    #将自变量矩阵增加一列全1元素,以便于截距项的计算
    x=np.column_stack((x,np.ones(n)))
    
    #利用cholesky分解求取回归方程的参数beta的估计值  
    s=np.array([lam]*p).tolist()
    s.insert(0,0)
    V=np.dot(x.T,x)+mat(diag(s))
    U=np.dot(x.T,y)
    R = np.linalg.cholesky(V)
    M=mforwardsolve(R, U)  
    mbacksolve(R.T,M)

### k折交叉验证

In [119]:
#给每一行数据一个组号，训练时用非组号的数据，测试时用该组号的数据
#用于训练岭回归的函数
def k_ridge(i,lam,x,y,index):
    if i == index:
        ridgereg(lam,np.delete(x, [i], axis=0),np.delete(y, [i]))#给每一行数据一个组号

In [123]:
#十折交叉验证
def ten_cvridgeregerr(lam,x,y):
    n=np.shape(x)[0]#n是行数
    index=random.sample((np.arange(10).tolist()*math.ceil(n/10)),n)#是每一行被分到第几组
    
    #矩阵中的元素作为当前删去折数作为参数传入k_ridge，结果第i行为删去第i折的岭回归系数估计值
    coe=pd.DataFrame(np.arange(1,11)).apply(k_ridge,lam=lam,x=x,y=y,index=index,axis=0)
    x=np.hstack(np.ones(n,1),x)
    mse=np.mean((np.sum(x*coe,axis=1)-y)**2)
    return mse

In [109]:
# ###test
# test_x=np.random.normal(loc=0.0, scale=1, size=(100,100))
# test_y=np.zeros(100)
# LAM_1=np.arange(0,1,0.01)

### 生成数据集 

In [ ]:
# #小数据集举例理解生成函数
# mean = [0, 0, 0]
# cov = [[1, 0.9,0.81], [0.9,1,0.729],[0.81,0.729,1]]
# x= np.random.multivariate_normal(mean, cov, 5)

In [104]:
#生成x
mean=np.zeros(1000)
cov =[]
i=0
while i < 1000:
    cov.append(0.9**(np.arange(1000)+i))
    i=i+1
for i in range(1000):
    cov[i][i]=1
x= np.random.multivariate_normal(mean, cov, 100000)

In [105]:
x.shape

(100000, 1000)

In [131]:
#生成用于生成真实y的β
beta0=[]
i=1
while i <=1000:
    if i%2 !=0:
        beta0.append(1)
    if i%2 ==0:
        beta0.append(-1)
    i=i+1
#生成y
y=np.dot(x,beta0)+np.random.randn(100000)

In [132]:
y.shape

(100000,)

In [134]:
#另一种生成beta0
# from itertools import chain
# a=np.array([1]*int(n/2)).reshape(int(n/2),1)
# b=-1*a
# betaTrue=list(chain(*np.hstack((a,b))))

### 交叉验证岭回归 选lambda 

In [137]:
#交叉验证100次 零到一之间（割线法/隔点法）选lamda
#割线法
def gexian():
    LAM_1=np.arange(0,1,0.01)
    mi=100000
    goodlam=0
    for i in LAM_1:
        mse=ten_cvridgeregerr(i,x,y)
        if mse<mi:
            mi=mse
            goodlam=i
    return mi,goodlam

In [148]:
#两点法
def liangdian():
    a=0
    b=1
    mi=100000
    m=(a+b)/2
    for j in range(100):#二分100次
        mse_m=ten_cvridgeregerr(m,x,y)
        mse_a=ten_cvridgeregerr(a,x,y)
        mse_b=ten_cvridgeregerr(b,x,y)
        if mse_a<mse_b:
            a=a,b=m
        if mse_a>=mse_b:
            a=m,b=b
    m=(a+b)/2
    mi=mse_m#因为是凸函数所以肯定越来越小不用比较,最后求一次均方误差就好，肯定快
                
    return m,mi#m就是选定的lambda
    